In [88]:
import pandas as pd
import numpy as np
import json
import spacy
from collections import Counter
import random

nlp = spacy.load("en_core_web_sm")

In [2]:
ds = pd.read_json('/Users/francisco/Library/CloudStorage/GoogleDrive-fzanartua@gmail.com/My Drive/Socialz/COBOL/all_cobol_events_2011_2021.json', lines=True)
ds['type'].value_counts()

PushEvent                        18959
PullRequestEvent                  7584
WatchEvent                        6692
IssueCommentEvent                 5061
CreateEvent                       4663
IssuesEvent                       4488
ForkEvent                         4118
PullRequestReviewCommentEvent     3294
PullRequestReviewEvent            2738
DeleteEvent                       1634
GollumEvent                        915
ReleaseEvent                       288
MemberEvent                        201
CommitCommentEvent                 181
PublicEvent                         25
TeamAddEvent                         4
Name: type, dtype: int64

In [3]:
ds['type'].value_counts().sum()

60845

In [4]:
ds['type'].value_counts().count()

16

In [5]:
ds['year'] = ds['created_at'].dt.year
pivot_table = ds.groupby(['type','year'])['created_at'].count()
pivot_table.reset_index().pivot('type','year')

created_at                                      \
year                                2011  2012   2013    2014   2015    2016   
type                                                                           
CommitCommentEvent                   NaN   NaN    NaN    62.0   23.0    37.0   
CreateEvent                          NaN  16.0   54.0    96.0   84.0   120.0   
DeleteEvent                          NaN   1.0    9.0    28.0   26.0    35.0   
ForkEvent                           10.0  13.0   14.0    79.0  150.0   184.0   
GollumEvent                          NaN   1.0    3.0     8.0  211.0   354.0   
IssueCommentEvent                    1.0   NaN    3.0   106.0  298.0   447.0   
IssuesEvent                          1.0   NaN    NaN    76.0  429.0   467.0   
MemberEvent                          NaN   2.0    8.0     6.0    8.0    11.0   
PublicEvent                          NaN   NaN    NaN     NaN    1.0     2.0   
PullRequestEvent                     4.0   NaN   15.0   169.0  307.0   357.0   
PullRequestReviewCommentEvent        NaN   NaN    NaN     NaN   34.0    13.0   
PullRequestReviewEvent               NaN   NaN    NaN     NaN    NaN     NaN   
PushEvent                            4.0  50.0  339.0  1685.0  896.0  1260.0   
ReleaseEvent                         NaN   NaN    NaN     1.0   15.0    18.0   
TeamAddEvent                         NaN   NaN    NaN     4.0    NaN     NaN   
WatchEvent                           6.0   8.0   13.0    65.0  497.0   617.0   

                                                                            
year                             2017    2018    2019    2020    2021 2022  
type                                                                        
CommitCommentEvent               13.0    14.0     5.0    17.0    10.0  NaN  
CreateEvent                     636.0   987.0   681.0  1105.0   884.0  NaN  
DeleteEvent                     139.0   254.0   261.0   466.0   415.0  NaN  
ForkEvent                       414.0   502.0   656.0  1205.0   890.0  1.0  
GollumEvent                     126.0   119.0    23.0    31.0    39.0  NaN  
IssueCommentEvent               433.0   441.0   446.0  1237.0  1647.0  2.0  
IssuesEvent                     807.0   577.0   446.0   566.0  1119.0  NaN  
MemberEvent                      18.0    22.0    22.0    67.0    37.0  NaN  
PublicEvent                       NaN     2.0     5.0     7.0     8.0  NaN  
PullRequestEvent                643.0  1104.0   997.0  1908.0  2080.0  NaN  
PullRequestReviewCommentEvent    42.0   200.0   740.0   825.0  1440.0  NaN  
PullRequestReviewEvent            NaN     NaN     NaN   534.0  2204.0  NaN  
PushEvent                      2472.0  2184.0  2170.0  4253.0  3646.0  NaN  
ReleaseEvent                    110.0    68.0    16.0    36.0    24.0  NaN  
TeamAddEvent                      NaN     NaN     NaN     NaN     NaN  NaN  
WatchEvent                      443.0   486.0   516.0  2964.0  1077.0  NaN

In [6]:
ds = ds.loc[ds['type'].isin(['ForkEvent', 'PullRequestEvent','PullRequestReviewCommentEvent', 'PullRequestReviewEvent', 'PushEvent','WatchEvent'])]

ds['payload'] = ds['payload'].apply(json.loads)

ds['title'] = ds['payload'].apply(lambda x: x.get('pull_request').get('title') if type(x.get('pull_request')) is dict else None)
ds['body'] = ds['payload'].apply(lambda x: x.get('pull_request').get('body') if type(x.get('pull_request')) is dict else None)
ds['message'] = ds['payload'].apply(lambda x: ' '.join([x.get('message') for x in x.get('commits')]) if type(x.get('commits')) is list else None)

In [7]:
corpus = Counter()


NoneType = type(None)


#corpus = []

column_names = ['title', 'body', 'message']

for column_name in column_names:
    column = ds[column_name]
    for text in column:
        if not isinstance(text, NoneType):
            for word in text.split():
                word = word.lower()
                corpus[word] += 1
                # if word not in corpus:
                #     corpus.extend([word])

#leave out words that appear less than two times.
corpus = [word[0] for word in corpus.most_common() if word[1] > 2]

In [8]:
# leave out words that are non ascii character, are digits, url, emails or puncts.
corpus = ' '.join(word for word in corpus)
corpus = list(set(token.text for token in nlp(corpus) if token.is_ascii 
                                        and not token.like_num 
                                        and not token.like_url 
                                        and not token.like_email 
                                        and not token.is_punct 
                                        and not token.is_digit))

In [9]:
len(corpus)

8451

In [19]:
ds['title_length'] = ds['title'].apply(lambda x: len(x) if not isinstance(x, NoneType) and len(x) > 0 else None)
ds['body_length'] = ds['body'].apply(lambda x: len(x) if not isinstance(x, NoneType) and len(x) > 0 else None)
ds['message_length'] = ds['message'].apply(lambda x: len(x) if not isinstance(x, NoneType) and len(x) > 0 else None)

ds['title_newlines'] = ds['title'].apply(lambda x: x.count('\n') if not isinstance(x, NoneType) and len(x) > 0 else None)
ds['body_newlines'] = ds['body'].apply(lambda x: x.count('\n') if not isinstance(x, NoneType) and len(x) > 0 else None)
ds['message_newlines'] = ds['message'].apply(lambda x: x.count('\n') if not isinstance(x, NoneType) and len(x) > 0 else None)
ds

,type,public,payload,repo,actor,org,created_at,id,other,year,title,body,message,title_length,body_length,message_length,title_newlines,body_newlines,message_newlines
0,PushEvent,True,"{'push_id': 3265220345, 'size': 1, 'distinct_s...","{'id': '164815300', 'name': 'blreay/test', 'ur...","{'id': '10861187', 'login': 'blreay', 'gravata...",{},2019-02-01 07:00:59+00:00,8.990824e+09,"{""actor"":{""display_login"":""blreay""}}",2019,None,None,test std::list,NaN,NaN,14.0,NaN,NaN,0.0
1,PushEvent,True,"{'push_id': 3452494485, 'size': 1, 'distinct_s...","{'id': '177935861', 'name': 'janilisch/CobolGr...","{'id': '48919728', 'login': 'CobViktor', 'grav...",{},2019-03-29 10:06:11+00:00,9.339725e+09,"{""actor"":{""display_login"":""CobViktor""}}",2019,None,None,todo anfuehrungszeichen,NaN,NaN,23.0,NaN,NaN,0.0
2,PushEvent,True,"{'push_id': 4210778419, 'size': 1, 'distinct_s...","{'id': '211251585', 'name': 'gigglegile/MavenS...","{'id': '33651173', 'login': 'gigglegile', 'gra...",{},2019-10-30 02:20:34+00:00,1.074348e+10,"{""actor"":{""display_login"":""gigglegile""}}",2019,None,None,添加findbug plugin,NaN,NaN,16.0,NaN,NaN,0.0
3,PushEvent,True,"{'push_id': 4092238228, 'size': 1, 'distinct_s...","{'id': '171827886', 'name': 'krisds/koopa', 'u...","{'id': '4458144', 'login': 'krisds', 'gravatar...",{},2019-09-30 10:13:31+00:00,1.052312e+10,"{""actor"":{""display_login"":""krisds""}}",2019,None,None,Updated usage text,NaN,NaN,18.0,NaN,NaN,0.0
5,PullRequestEvent,True,"{'action': 'opened', 'number': 2, 'pull_reques...","{'id': '165220205', 'name': 'tagadatsointsoin/...","{'id': '46569692', 'login': 'tagadatsointsoin'...",{},2019-05-15 07:39:12+00:00,9.628609e+09,"{""actor"":{""display_login"":""tagadatsointsoin""}}",2019,My branch,,None,9.0,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60840,PushEvent,True,"{'push_id': 2498649239, 'size': 2, 'distinct_s...","{'id': '129782940', 'name': 'RosesTheN00b/grad...","{'id': '5215946', 'login': 'RosesTheN00b', 'gr...",{},2018-04-19 13:11:47+00:00,7.555460e+09,"{""actor"":{""display_login"":""RosesTheN00b""}}",2018,None,None,fixed log paths new version number :-),NaN,NaN,38.0,NaN,NaN,0.0
60841,PushEvent,True,"{'push_id': 2665217304, 'size': 1, 'distinct_s...","{'id': '129782940', 'name': 'RosesTheN00b/grad...","{'id': '5215946', 'login': 'RosesTheN00b', 'gr...",{},2018-06-22 09:47:38+00:00,7.863185e+09,"{""actor"":{""display_login"":""RosesTheN00b""}}",2018,None,None,fixed gnu cobol 3 formatting,NaN,NaN,28.0,NaN,NaN,0.0
60842,PushEvent,True,"{'push_id': 2631811764, 'size': 1, 'distinct_s...","{'id': '129782940', 'name': 'RosesTheN00b/grad...","{'id': '5215946', 'login': 'RosesTheN00b', 'gr...",{},2018-06-09 07:55:39+00:00,7.800320e+09,"{""actor"":{""display_login"":""RosesTheN00b""}}",2018,None,None,new version :-)!\n!,NaN,NaN,18.0,NaN,NaN,1.0
60843,PushEvent,True,"{'push_id': 2498652724, 'size': 1, 'distinct_s...","{'id': '129782940', 'name': 'RosesTheN00b/grad...","{'id': '5215946', 'login': 'RosesTheN00b', 'gr...",{},2018-04-19 13:12:56+00:00,7.555467e+09,"{""actor"":{""display_login"":""RosesTheN00b""}}",2018,None,None,updated version number,NaN,NaN,22.0,NaN,NaN,0.0


In [20]:
stats = pd.concat([ds['title_newlines'].describe(), ds['body_newlines'].describe(), ds['message_newlines'].describe()], axis=1)
stats

,title_newlines,body_newlines,message_newlines
count,13616.0,8594.000000,16825.000000
mean,0.0,6.862462,1.719287
std,0.0,13.971748,9.858037
min,0.0,0.000000,0.000000
25%,0.0,0.000000,0.000000
50%,0.0,2.000000,0.000000
75%,0.0,8.000000,2.000000
max,0.0,131.000000,352.000000


In [138]:
stats = pd.concat([ds['title_length'].describe(), ds['body_length'].describe(), ds['message_length'].describe()], axis=1)
stats

,title_length,body_length,message_length
count,13616.000000,8594.000000,16825.000000
mean,32.070138,460.007796,127.498603
std,17.988142,912.830115,372.472139
min,1.000000,1.000000,1.000000
25%,18.000000,42.000000,19.000000
50%,29.000000,157.000000,41.000000
75%,42.000000,437.000000,100.000000
max,117.000000,10679.000000,10211.000000


In [12]:
# Random title
n = random.randint(18,42)
print(n)
' '.join(random.choices(corpus, k=n))

36


'miissing compilerdirectivebuilder transfer @paulnewt programsymboltablebuilder make lwmlddl revisando __l performences href="https://www.npmjs.com/~mathias">mathias</a placeholder eu default_justify summit cobol85checker bits break vip gain way depending vs.-test scratch fetches aetna 1939_improveexpressiontrees remarks boolean valida updated runrtbsam instruct anyways inform improving'

In [13]:
# Random body
n = random.randint(42,437)
print(n)
' '.join(random.choices(corpus, k=n))

44


'boxing err persist id reachability datacategory testbranch6 custom_2_changes tail carlosalbertofigueiredo proc::check_args errs lt;artifactid&gt;guava&lt;/artifactid&gt gerald hiring debut happening categorized blog during intel $ textinput rc direct reversed parseunsafe projectiles due v1.6.00 ^^ d2x matt endevor con llvm moving typecobol_version l -d tweaking md2 zcobol\\opt rpn'

In [14]:
# Random message
n = random.randint(19,100)
print(n)
' '.join(random.choices(corpus, k=n))

39


'artifactory representation sesitivity v4.17.18</li strongly leading cms 0ct spreadsheet detecting usados div25singh bug#94414 kept python 6.5.3</li c29dd8239d6b353e699397eb090a1fd27411fa24"><code acorn symboluser stabilize failover.</p max msd ready java13 ibm generation aps drive co*97 minimized predecessors sonarexample precompile set_timeout helloworld towards 1452_warningendalignment chance'

In [18]:
with open('Data/Corpus.txt','w') as fp:
    for word in corpus:
        # write each item on a new line
        fp.write("%s\n" % word)
    print('Done')

Done
